## get reviews details about a doctor
### the methodology:
#### for every doctor - search and find the residential using the npi key
#### for every doctor name and location - search and find reviews

#### some obstacles in the way :
#### 1. a lot of websites have protection against web-scraping.
#### 2. not every doctor have review in the website you search.

In [234]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [305]:
# upload initial data
initial_data_df = pd.read_excel(r'C:\Users\nirro\Downloads\deloitte\NPI_initial.xlsx')
initial_data_df['first_and_last'] = initial_data_df['First Name'].values + ' ' +initial_data_df['Last Name'].values
# initial_data_df['location'] = 'us'
initial_data_df.head(5)

,Last Name,First Name,NPI,first_and_last
0,EARNEST,MELISSA,1023154903,MELISSA EARNEST
1,Jones,Jason,1548430291,Jason Jones


### get location of doctor by npi number 

In [307]:
d = webdriver.Chrome()
npidict = {}

for npi in initial_data_df['NPI'].values:
    d.get('https://www.hipaaspace.com/')
    # search using first and last names
    e = d.find_element(By.ID, 'tbxSearchRequest') 
    e.send_keys(str(npi))

    e.send_keys(Keys.ENTER)
    time.sleep(1)
    e = d.find_element(By.XPATH,"//div[@style='margin: 0px 0px 10px 15px']").text.replace('\n',',').split(',')
    
    time.sleep(1)
    npidict[npi] = f'{e[1]} {e[2]}'

print(npidict)
d.quit()


{1023154903: 'MENOMONEE FALLS  WI', 1548430291: 'NASHVILLE  TN'}


In [ ]:
#add column location
initial_data_df['location'] = [npidict[npi] for npi in initial_data_df['NPI'].values]
initial_data_df.head(5)

### add a new list (doctor with reviews)

In [312]:

# add new row for reference
df_new_row =pd.DataFrame({'Last Name':'Tarango','First Name':'Sonia','NPI':'1111111111','first_and_last':'Sonia Tarango','location':'San Antonio, TX'},index=[0])
initial_data_df = initial_data_df.append(df_new_row)
initial_data_df.head(5)

,Last Name,First Name,NPI,first_and_last,location
0,EARNEST,MELISSA,1023154903,MELISSA EARNEST,MENOMONEE FALLS WI
1,Jones,Jason,1548430291,Jason Jones,NASHVILLE TN
0,Tarango,Sonia,111111111,Sonia Tarango,"San Antonio, TX"
0,Tarango,Sonia,1111111111,Sonia Tarango,"San Antonio, TX"


###  find a doctor urls using name and location

In [313]:
# using search bar to find a doctor reviews

d = webdriver.Chrome()
# names = ['Sonia Tarango','EARNEST MELISSA']
# locations =['San Antonio, TX','us']
urlist = []
# for name,locat in zip(names,locations):
for name,locat in zip(initial_data_df['first_and_last'].values,initial_data_df['location'].values):  
    d.get('https://www.caredash.com')
    # search using first and last names
    e = d.find_element(By.ID, 'providers_search_query') 
    e.send_keys(name)
    
    # search using location
    e = d.find_element(By.ID, 'providers_search_location') 
#     e2.send_Keys(Keys.chord(Keys.CONTROL, "a"), "55")
    e.clear()
    e.send_keys(locat)
    e.send_keys(Keys.ENTER)

#     print(d.current_url)
#     d.find_element(By.CLASS_NAME,"srl-item-name no-margin").Click()
    delay = 20 # seconds
    time.sleep(5)
    e3 = d.find_element(By.XPATH,"//span[@class='cd1-name']")
    e3.click()
    time.sleep(5)
#     new_url = str(d.current_url)
    window_new = d.window_handles[-1]
    d.switch_to.window(window_new)
    new_url = str(d.current_url)
    print(f'doctor {name} url is: {new_url}')
    urlist.append(new_url)
    time.sleep(5)
d.quit()


doctor MELISSA EARNEST url is: https://www.caredash.com/doctors/melissa-earnest-md-menomonee-falls-wi
doctor Jason Jones url is: https://www.caredash.com/doctors/jason-jones-md-nashville-tn
doctor Sonia Tarango url is: https://www.caredash.com/doctors/sonia-tarango-lfswd
doctor Sonia Tarango url is: https://www.caredash.com/doctors/sonia-tarango-lfswd


### get reviews for each doctor

In [322]:
# read reviews of doctor
for doc_url in urlist:

    html_request = requests.get(doc_url).text
    # print(html_request)

    soup = BeautifulSoup(html_request,'lxml')
    # print(soup)
    # get review details

    list_reviewer_detail = soup.find_all('div',class_='review-item-name')
#     print(list_reviewer_detail)
    list_reviews_detail_final = []
    for review_detail in list_reviewer_detail:
        reviewer_date = review_detail.find('p',class_='reviewer-date no-margin').text.replace('\n',"")
        reviewer_name = review_detail.find('p',class_='reviewer-name no-margin text-truncate').text.replace('\n',"").replace(' ',"").strip()
        reviewer_verified = review_detail.find('p',class_='reviewer-status no-margin').text.replace('\n',"")
        reviewer_number_reviews = review_detail.find('p',class_='reviewer-number no-margin').text.replace('\n',"")
        reviewer_location = review_detail.find('p',class_='reviewer-address no-margin').text.replace('\n',"")

    #     print(f'date: {reviewer_date}')
    #     print(f'name: {reviewer_name}')
    #     print(f'verification: {reviewer_verified}')
    #     print(f'number of reviews: {reviewer_number_reviews}')
    #     print(f'location: {reviewer_location}')
        list_reviews_detail_final.append((reviewer_date,reviewer_name,reviewer_verified,reviewer_number_reviews,reviewer_location))
    if len(list_reviews_detail_final) ==0:
        list_reviews_detail_final.append('no reviews')
    print(list_reviews_detail_final)
    
    # get review details
    review_card_list = soup.find_all('div',class_='col-sm-9')
    list_card_final = []
    for card in review_card_list:

        number_total_stars_for_review = len(revieww_card.find('div',class_='listing-star-rating overall-rating mt-0').find_all('span',class_='icon-star-full'))

    #     print(f'total score of review: {number_total_stars_for_review}')

        review_text = card.find('div', class_='expander-txt mt-review-body-readmore review-body-text').text.replace('\n','')
    #     print(f'review text: {review_text}')
        list_star_scores_for_review = card.find('div',class_='subrating-content')
        list_scores = list_star_scores_for_review.find_all('p')
        d_scores={}
        for score in list_scores:
            score_text = score.text.replace('\n',"").replace('(2)',"")
    #         print(score_text)
            score_rate = len(score.find_all('span',class_='icon-star-full'))
    #         print(score_rate)
            d_scores[score_text] = score_rate
        wait_time = card.find('p',class_='subrating-content').text.replace('\n','')
    #     print(wait_time)
        
        list_card_final.append((number_total_stars_for_review,review_text,d_scores,wait_time))
    overall_score = soup.find('p',class_='mb-0 overall-rating-value').text.replace('\n','')
    
    list_card_final.append(overall_score)
    if len(list_card_final) ==0:
        list_card_final.append('no reviews')
    print(list_card_final)
    print('--------------------')

[('July 15, 2019', 'LKC', 'Verified Reviewer', '1 Review', 'San Antonio, TX')]
[(4, 'I meet Dr. Sonia Tarango at the University Heath Clinic and I feel that we will get along good as a patient and doctor meeting . But it was hard to get her to refill a medication that I had to have .', {'Communication': 5, 'Scheduling': 5, 'Staff': 5, 'Treatment': 5, 'Bedside Manner': 4}, 'Wait Time: 15 to 30 minutes'), (4, 'She was very helpful and nice', {'Communication': 5, 'Scheduling': 5, 'Staff': 5, 'Treatment': 5, 'Bedside Manner': 5}, 'Wait Time: 31 to 45 minutes'), '3.6']
--------------------
[('July 15, 2019', 'LKC', 'Verified Reviewer', '1 Review', 'San Antonio, TX'), ('May 10, 2019', 'Deb', 'Verified Reviewer', '1 Review', 'San Antonio, TX')]
[(4, 'I meet Dr. Sonia Tarango at the University Heath Clinic and I feel that we will get along good as a patient and doctor meeting . But it was hard to get her to refill a medication that I had to have .', {'Communication': 5, 'Scheduling': 5, 'Staff